In [367]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [368]:
# Plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.tools as tls

In [369]:
# Set-up Plotly
tls.set_credentials_file(username="jclasul", api_key="MKXHNoKq9cRqNshsRQlh")
tls.set_config_file(sharing="public")

In [370]:
HR = pd.read_csv("HRIN.csv")

In [371]:
# 35 variables for 1470 persons
HR.shape

(1470, 35)

In [390]:
# List all variables and data types
HR.dtypes.value_counts()

int64     26
object     9
dtype: int64

In [396]:
# Dealing with CATEGORICAL variables
# # # # # # # # # # # #
# Method 1:
# One-Hot encoding
# # # # # # # # # # # #

recast = pd.DataFrame.dropna(pd.get_dummies(HR, sparse=True), axis=[0,1],how="all")
recast_corr = pd.DataFrame.dropna(recast.corr(), axis=[0,1],how="all")

# Plotly CONTOUR map
# 
# values have to be in LIST for PLOTLY, convert tolist and use COLUMN names for axis
# 

py.iplot([go.Contour(z=recast_corr.values.tolist(),
                   x=recast.columns.tolist(),
                   y=recast.columns.tolist())])

In [395]:
# # # # # # # # # # # #
# Method 2:
# Label Encoding
# # # # # # # # # # # #
#
# Convert all Object dtypes to Categorical

def TO_CAT(col):
    return col.astype("category").cat.codes
    
encoding = HR.apply(TO_CAT, axis=0)
encoding_corr = pd.DataFrame.dropna(encoding.corr(), axis=[0,1],how="all")

# Plotly CONTOUR map
# 
# values have to be in LIST for PLOTLY, convert tolist and use COLUMN names for axis
# 

py.iplot([go.Contour(z=encoding_corr.values.tolist(),
                   x=encoding.columns.tolist(),
                   y=encoding.columns.tolist())])

In [405]:
# START machine learning
# something
# something else
# finish

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedShuffleSplit

In [ ]:
sns.pairplot(encoding_corr, hue="Attrition")

KeyboardInterrupt: 

In [ ]:
# SPLIT data
train, test, target_train, target_val = train_test_split(attrition_final, target, train_size= 0.75,random_state=0);